# Import all tables for the eICU-CRD（eICU Collaborative Research Database）
## 由于机器内存的限制，无法导入eICU的所有表，因此在后续逐个导入表，下面注释的代码是使用动态字典导入的方法。

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

path = 'G:/eicu-crd'
## Gets all filenames ending in .csv
# all_files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f.endswith('.csv')]
## The data is loaded dynamically using a dictionary, with the file name as the key
# dfs = {}
# for file_name in all_files:
    # df_name = file_name.split('.')[0]
    # dfs[df_name] = pd.read_csv(os.path.join(path, file_name), low_memory=False)
    # print(f'{file_name} imported')

# Demographics of the patients
## 对eICU进行患者特征的提取，具体来说，从patient、apachepatientresult和note这三个表中提取和计算相关的特征。
+ Gender: 代表患者的性别。某些疾病在男性和女性之间的发病率不同（patient表中的gender）。
+ Age: 代表患者的年龄。它对许多疾病的发病率、预后和治疗选择有重要意义（patient表中的age）。
+ Ethnicity: 代表患者的种族或民族背景。某些疾病可能在不同的种族或族群中有不同的发病率（patient表中的ethnicity）。
+ Height: 代表患者的身高。对药物剂量、麻醉、手术风险等有关（patient表中的admissionheight）。
+ Hospital length of stay in days: 患者在医院住院的天数，用于衡量患者疾病的严重性、复杂性以及所需的医疗干预。医院的住院天数反映了患者的疾病状态、治疗反应和康复进度。长时间的住院可能意味着患者的疾病较为严重、复杂或其治疗和恢复需要更长的时间。而短时间的住院可能意味着患者的疾病状况较为轻微或得到了及时和有效的治疗。因此，这个变量可以为医疗团队提供有关患者健康状况和治疗需要的重要信息（由patient表中的两个时间计算得出(hospitaldischargeoffset - hospitaladmitoffset)/60.0/24.0）。
+ Source of admission: 代表患者被转入的来源，例如从另一家医院、诊所或是家中。这可以提供关于患者之前状况的线索（patient表中的hospitaladmitsource）。
+ Hospital discharge location: 代表患者出院后被转送到的地点或设施，例如家庭、康复中心。出院地点可以反应患者出院时的健康状况、照护需求和恢复预期（patient表中的hospitaldischargelocation）。
+ Hospital discharge status: 代表患者出院时的状态，生存或者死亡（patient表中的hospitaldischargestatus）。
+ Unit type: 代表患者进入单元的类型（patient表中unittype）。
+ Unit admit source: 代表患者被转入单元的来源（patient表中的unitadmitsource）。
+ Weight: 代表患者的体重。对药物剂量、麻醉、手术风险等有关（patient表中的admissionweight或者dischargeweight）。
+ Unit length of stay in days: 患者在单元住院的天数，用于衡量患者疾病的严重性、复杂性以及所需的医疗干预（patient表中的unitdischargeoffset，unitdischargeoffset / 60.0 / 24.0）。
+ Unit discharge status: 代表患者出单元时的状态，生存或者死亡（patient表中的unitdischargestatus）。
+ APACHE score: 一种评估重症监护病人疾病严重度和预测其住院死亡率的工具。提供了关于患者在进入重症监护室时健康状况的深入了解。该分数考虑了多个生理测量值、年龄、慢性健康状况等因素，为医生提供了患者可能的治疗结果的预测（apachePatientResult表中的apachescore）。
+ Smoking: 代表患者是否吸烟。吸烟是多种疾病的危险因素（从note表中经过正则表达式推理得来）。

In [ ]:
# Import
patient_cols = [
    'patientunitstayid', 'gender', 'age', 'ethnicity', 'admissionheight',
    'hospitaldischargeoffset', 'hospitaladmitoffset', 'hospitaladmitsource',
    'hospitaldischargelocation', 'hospitaldischargestatus', 'unittype',
    'unitadmitsource', 'admissionweight', 'unitdischargeoffset', 'unitdischargestatus'
]
patient_df = pd.read_csv(path + '/patient.csv', usecols=patient_cols)
apachepatientresult_cols = ['patientunitstayid', 'apachescore', 'apacheversion']
apachepatientresult_df = pd.read_csv(path + '/apachePatientResult.csv', usecols=apachepatientresult_cols)
# Selecte Patients and scores with Apache version IV
apachepatientresult_df = apachepatientresult_df.query("apacheversion == 'IV'")[['patientunitstayid', 'apachescore']]
# Calculate length of stay and unit length of stay
patient_df['hospital_length_of_stay_in_days'] = (patient_df['hospitaldischargeoffset'] - patient_df['hospitaladmitoffset']) / 60.0 / 24.0
patient_df['unit_length_of_stay_in_days'] = patient_df['unitdischargeoffset'] / 60.0 / 24.0
# Transform the age of '> 89' to a numerical value 90
patient_df['age'] = patient_df['age'].replace('> 89', '90').apply(pd.to_numeric, errors='coerce')
patient_df = patient_df.drop(columns=['hospitaladmitoffset', 'hospitaldischargeoffset', 'unitdischargeoffset'])
# Merge with APACHE_score
merged_df = pd.merge(patient_df, apachepatientresult_df, on='patientunitstayid', how='left')

# Vital signs
## 对eICU进行生命体征提取。具体来说，从vitalaperiodic、vitalperiodic中提取和计算相关的特征。包括：
+ nonInvasiveSystolic: 患者无创收缩值（vitalAperiodic表中noninvasivesystolic）。real
+ nonInvasiveDiastolic: 患者无创舒张值（vitalAperiodic表中noninvasivediastolic）。real
+ noninvasivemean：患者BP平均值（vitalAperiodic表中noninvasivemean）。
+ PAOP (Pulmonary Artery Obstruction Pressure): 肺动脉阻塞压（vitalAperiodic表中paop）。real
+ temperature: 体温（vitalPeriodic表中temperature）。decimal(11,4)
+ sao2: 血氧饱和度（vitalPeriodic表中sao2）。int
+ heartrate: 心率（vitalPeriodic表中heartrate）。int
+ respiration: 呼吸频率（vitalPeriodic表中respiration）。int
+ cvp (central venous pressure): 中心静脉压（vitalPeriodic表中cvp）。int
+ etco2 (end-tidal carbon dioxide): 末梢呼气二氧化碳，医生通过监测患者的末梢呼气二氧化碳来评估他的呼吸状况（vitalPeriodic表中etco2）。int
+ ICP: 颅内压（vitalPeriodic表中icp）。int

In [ ]:
# Import
# datatype map
dtype_map = {
    'noninvasivesystolic': 'float32',
    'noninvasivediastolic': 'float32',
    'noninvasivemean':'float32',
    'paop': 'float32',
    'temperature': 'float32',
    'sao2': 'float32',
    'heartrate': 'float32',
    'respiration': 'float32',
    'cvp': 'float32',
    'etco2': 'float32',
    'icp': 'float32'
}
vitalaperiodic_cols = ['patientunitstayid', 'noninvasivesystolic', 'noninvasivediastolic', 'noninvasivemean', 'paop']
vitalaperiodic_df = pd.read_csv(path + '/vitalAperiodic.csv', usecols=vitalaperiodic_cols, dtype=dtype_map)

vitalaperiodic_df = vitalaperiodic_df.groupby('patientunitstayid').mean().reset_index()
vitalperiodic_cols = ['patientunitstayid', 'temperature', 'sao2', 'heartrate', 'respiration', 
                      'cvp', 'etco2', 'icp']
vitalperiodic_df = pd.read_csv(path + '/vitalPeriodic.csv', usecols=vitalperiodic_cols, dtype=dtype_map)

vitalperiodic_df = vitalperiodic_df.groupby('patientunitstayid').mean().reset_index()
# Merge
merged_df = pd.merge(merged_df, vitalaperiodic_df, on='patientunitstayid', how='left')
merged_df = pd.merge(merged_df, vitalperiodic_df, on='patientunitstayid', how='left')

# Glasgow Coma Scale (GCS)
## 在eICU数据库中，Glasgow Coma Scale（GCS）的数据通常可以在nurseCharting表中找到。这个表通常包括各种从护士记录中获取的数据，包括患者的GCS评分

In [ ]:
# Import
nursecharting_cols = ['patientunitstayid', 'nursingchartcelltypevallabel', 'nursingchartcelltypevalname', 'nursingchartvalue']
nursecharting_df = pd.read_csv(path + '/nurseCharting.csv', usecols=nursecharting_cols)
# Filter
gcs_related = ['Glasgow coma score']
filtered_df = nursecharting_df[nursecharting_df['nursingchartcelltypevallabel'].isin(gcs_related)]
# Numeric
filtered_df['nursingchartvalue'] = pd.to_numeric(filtered_df['nursingchartvalue'], errors='coerce')
# Group by and calculate the mean
grouped_df = filtered_df.groupby(['patientunitstayid']).mean().reset_index()
# Rename the columns
grouped_df.columns = ['patientunitstayid', 'gcsscore']
# Merge the DataFrames
merged_df = pd.merge(merged_df, grouped_df, on='patientunitstayid', how='left')

# Intake and Output Records (I&O or I/O)
## 在eICU数据库中，I&O 通常存储在intakeOutput表中。这个表包括与患者摄入和排放量有关的多种类型的信息，包括液体摄入、药物摄入以及各种类型的输出。
## 
## Intake
+ P.O.: 口服，即通过口腔进食或饮水。Generic Intake (ml)
+ NS IVF: Normal Saline Intravenous Fluid，即通过静脉注射输送的生理盐水。Generic Intake (ml)|NS IVF
+ Continuous infusion meds: 持续输液药物，即通过持续输液的方式给药。Crystalloids (ml)|Continuous infusion meds
## Output
+ Urine: 尿液，可能包括各种类型和情况下的尿液排放。Output (ml)|Urine
+ Stool: 粪便。Output (ml)|Stool

In [ ]:
# Import
intakeoutput_cols = ['patientunitstayid', 'cellpath', 'celllabel', 'cellvaluenumeric']
intakeoutput_df = pd.read_csv(path + '/intakeOutput.csv', usecols=intakeoutput_cols)
selected_labels = ['P.O.', 'NS IVF', 'Continuous infusion meds', 'Urine', 'Stool']
filtered_df = intakeoutput_df[intakeoutput_df['celllabel'].isin(selected_labels)]
# Average against each 'patientunitstayid' and 'celllabel'
average_values_df = filtered_df.groupby(['patientunitstayid', 'celllabel']).mean().reset_index()
# Use the pivot function to reshape the DataFrame
pivot_df = average_values_df.pivot(index='patientunitstayid', columns='celllabel', values='cellvaluenumeric')
# Reset the index to concatenate with merged_df
pivot_df.reset_index(inplace=True)
# Merge
merged_df = pd.merge(merged_df, pivot_df, on='patientunitstayid', how='left')

# Lab Result (The most common 20 lab results in eICU)
## 在eICU数据库中，lab result存储在lab表中。这个表包括患者的多个lab测试结果，包括葡萄糖、钾离子含量等。
+ bedside glucose（mg/dL）：病床旁测量的血糖值。
+ potassium（mmol/L）：血液中的钾浓度。
+ sodium（mmol/L）：血液中的钠浓度。
+ glucose（mg/dL）：血糖浓度。
+ Hgb（g/dL）：血红蛋白含量。
+ chloride（mmol/L）：血液中的氯浓度。
+ Hct（%）：红细胞压积或血容比。
+ creatinine（mg/dL）：肌酐浓度，用于评估肾功能。
+ BUN（mg/dL）：血尿素氮，另一种评估肾功能的指标。
+ calcium（mg/dL）：血液中的钙浓度。
+ bicarbonate（mmol/L）：血液中的碳酸氢盐浓度。
+ platelets x 1000（K/mcL）：血小板计数。
+ WBC x 1000（K/mcL）：白细胞计数。
+ RBC（M/mcL）：红细胞计数。
+ MCV（fL）：平均红细胞体积。
+ MCHC（g/dL）：平均红细胞血红蛋白浓度。
+ MCH（pg）：平均红细胞血红蛋白含量。
+ RDW（%）：红细胞分布宽度，描述红细胞大小的变异程度。
+ anion gap：阴离子间隙，用于诊断酸碱平衡异常。
+ MPV（fL）：平均血小板体积。

In [ ]:
# Import
lab_columns = ['patientunitstayid', 'labname', 'labresult', 'labmeasurenamesystem']
lab_df = pd.read_csv(path + '/lab.csv', usecols=lab_columns)
grouped_lab_df = lab_df.groupby(['patientunitstayid', 'labname']).mean().reset_index()
labname_counts = lab_df['labname'].value_counts()
labname_counts_sorted = labname_counts.sort_values(ascending=False)
top_20_labnames = labname_counts_sorted.head(20).index.tolist()
filtered_lab_df = lab_df[lab_df['labname'].isin(top_20_labnames)]
pivot_lab_df = filtered_lab_df.pivot_table(index='patientunitstayid', 
                                           columns='labname', 
                                           values='labresult', 
                                           aggfunc='mean').reset_index()
merged_df = pd.merge(merged_df, pivot_lab_df, on='patientunitstayid', how='left')

# Impute the missing values

In [ ]:
# Observe missing values
print(merged_df.isna().sum()/len(merged_df))

## Use values in nurseCharting table to impute temperature in merged_df

In [ ]:
# Temperature is missing by more than 90%. Temperature in nurseCharting is used for imputation.
# Filter for Temperature
temperature_condition = (nursecharting_df['nursingchartcelltypevallabel'] == 'Temperature') & \
                        (nursecharting_df['nursingchartcelltypevalname'].isin(['Temperature (F)', 'Temperature (C)']))
filtered_df = nursecharting_df[temperature_condition]
# Convert values to numeric
filtered_df['nursingchartvalue'] = pd.to_numeric(filtered_df['nursingchartvalue'], errors='coerce')
# Convert Fahrenheit to Celsius if needed
fahrenheit_condition = (filtered_df['nursingchartcelltypevalname'] == 'Temperature (F)')
filtered_df.loc[fahrenheit_condition, 'nursingchartvalue'] = (filtered_df.loc[fahrenheit_condition, 'nursingchartvalue'] - 32) * 5/9
# Group by and calculate the mean
grouped_df = filtered_df.groupby(['patientunitstayid']).mean().reset_index()
# Rename the columns
grouped_df.columns = ['patientunitstayid', 'temperature']
# Assuming that 'temperature' is the column in merged_df containing the existing temperature data
merged_df['temperature'] = pd.to_numeric(merged_df['temperature'], errors='coerce')
# Merge the two DataFrames on 'patientunitstayid' using an outer join to keep all records
merged_temp_df = pd.merge(merged_df[['patientunitstayid', 'temperature']], 
                          grouped_df, on='patientunitstayid', how='outer', suffixes=('_merged', '_grouped'))
# Calculate the new 'temperature' column
merged_temp_df['temperature'] = np.where(
    merged_temp_df['temperature_merged'].isna(), 
    merged_temp_df['temperature_grouped'], 
    np.where(
        merged_temp_df['temperature_grouped'].isna(), 
        merged_temp_df['temperature_merged'], 
        (merged_temp_df['temperature_merged'] + merged_temp_df['temperature_grouped']) / 2
    )
)
# Drop the temporary columns
merged_temp_df.drop(['temperature_merged', 'temperature_grouped'], axis=1, inplace=True)
# Merge this back into the original DataFrame
merged_df = pd.merge(merged_df.drop('temperature', axis=1), merged_temp_df, on='patientunitstayid', how='left')

In [ ]:
# Observe missing values
print(merged_df['temperature'].isna().sum()/len(merged_df))

In [ ]:
# Export merged_df to csv for missing value visualization and data imputation using R studio
merged_df.to_csv(path + '/merged_df.csv', index=False)